In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np 
import pandas as pd 
from scipy import signal
import matplotlib.pyplot as plt

# Import RandomForest Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import metrics

from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [2]:
stock = pd.read_csv('/Users/k0s00ks/Documents/iRage Capital/Data Science Assignment (Latest)/s1.csv')

# Convert the Date column to Date and time seperately
stock['date'] = pd.to_datetime(stock['date'])
stock['Time'],stock['Date']= stock['date'].apply(lambda x:x.time()), stock['date'].apply(lambda x:x.date())
stock['Hour'] = stock['date'].apply(lambda x:x.hour)

In [3]:
cols = stock.columns
cols.drop('30secAhead', '1minAhead')
stock['Date'] = pd.to_datetime(stock['Date'])
y_cols = ['30secAhead', '1minAhead', 'Date', 'Time', 'date']
X, y = stock[cols], stock[y_cols]
X_train, X_test = X.loc[X['Date'] < pd.to_datetime('2016-10-15', format = '%Y-%m-%d' )], X.loc[X['Date'] >= pd.to_datetime('2016-10-15', format = '%Y-%m-%d' )]
y_train, y_test = y.loc[y['Date'] < pd.to_datetime('2016-10-15', format = '%Y-%m-%d' )], y.loc[y['Date'] >= pd.to_datetime('2016-10-15', format = '%Y-%m-%d' )]

# Model Type 1: Random Forest Regression
Now we start to build models to predict the 30 second change. 

In [6]:
model_cols_X = ['b0', 'a0', 'bsz0', 'bidx0', 'sidx0', 'ssz0', 'bsz1', 'bidx1', 
                'sidx1', 'ssz1', 'bsz2', 'bidx2', 'sidx2', 'ssz2', 'bsz3', 'bidx3', 
                'sidx3', 'ssz3', 'bsz4', 'bidx4', 'sidx4', 'ssz4', 'f25', 'f26', 
                'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 
                'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 
                'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 
                'f57', 'f58', 'b0_cm', 'a0_cm', 'bsz0_cm', 'bidx0_cm', 'sidx0_cm', 'ssz0_cm',
                'bsz1_cm', 'bidx1_cm', 'sidx1_cm', 'ssz1_cm', 'bsz2_cm', 'bidx2_cm', 'sidx2_cm',
                'ssz2_cm', 'bsz3_cm', 'bidx3_cm', 'sidx3_cm', 'ssz3_cm', 'bsz4_cm', 'bidx4_cm', 
                'sidx4_cm', 'ssz4_cm', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 
                'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 
                'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 
                'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 
                'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 
                'f131', 'f132', 'f133', 'f134', 'f139', 'f140', 'f141', 'f142', 'f143', 'f144', 
                'f145', 'f146', 'f147', 'f148','Hour']

model_cols_y = ['30secAhead', '1minAhead']

rfr0 = RandomForestRegressor(n_jobs=-1, verbose=1)
rfr0.fit(X_train[model_cols_X], y_train['30secAhead'])

imporatnce_matrix_0 = pd.DataFrame(0, index = X_train[model_cols_X].columns, columns = ['Importance'])
imporatnce_matrix_0['Importance'] = rfr0.feature_importances_
print (imporatnce_matrix_0.sort('Importance', ascending=False).head(20).index)

train_accuracy_rfr0 = rfr0.score(X_train[model_cols_X], y_train['30secAhead'])
test_accuracy_rfr0 = rfr0.score(X_test[model_cols_X], y_test['30secAhead'])

print ('Test accuracy is : ' + str(test_accuracy_rfr0) + ' and train accuracy is : ' + str(train_accuracy_rfr0))

[Parallel(n_jobs=8)]: Done   1 out of   8 | elapsed: 10.5min remaining: 73.2min
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed: 10.8min finished
/Users/k0s00ks/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:25: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


Index([u'bidx0', u'sidx0', u'bidx3', u'f25', u'f129', u'bidx1', u'bsz0',
       u'f85', u'f145', u'f146', u'f56', u'f128', u'f148', u'f114', u'f113',
       u'f112', u'f55', u'f130', u'bidx4', u'f121'],
      dtype='object')


[Parallel(n_jobs=8)]: Done   1 out of   8 | elapsed:    2.0s remaining:   14.1s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    6.4s finished
[Parallel(n_jobs=8)]: Done   1 out of   8 | elapsed:    0.6s remaining:    3.9s


Test accuracy is : -0.647607499446 and train accuracy is : 0.937304438359


[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    2.3s finished


We see that though score on train data is coming out to be awesome, the result on data fails horribly. The RandomForest model is currently overfitting. We may try the model with less variables.
Using the importance matrix we select the top 20 most important variables for building the ensemble of trees.

# MODEL 2:

In [7]:
rfr1 = RandomForestRegressor(n_jobs=-1, verbose=1)
rfr1.fit(X_train[imporatnce_matrix_0.sort('Importance', ascending=False).head(20).index], y_train['30secAhead'])

train_accuracy_rfr1 = rfr1.score(X_train[imporatnce_matrix_0.sort('Importance', ascending=False).head(20).index], y_train['30secAhead'])
test_accuracy_rfr1 = rfr1.score(X_test[imporatnce_matrix_0.sort('Importance', ascending=False).head(20).index], y_test['30secAhead'])

print ('Test accuracy is : ' + str(test_accuracy_rfr1) + ' and train accuracy is : ' + str(train_accuracy_rfr1))

/Users/k0s00ks/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app
[Parallel(n_jobs=8)]: Done   1 out of   8 | elapsed:  1.4min remaining:  9.8min
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:  1.4min finished
/Users/k0s00ks/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
[Parallel(n_jobs=8)]: Done   1 out of   8 | elapsed:    0.6s remaining:    4.4s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    1.4s finished
/Users/k0s00ks/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
[Parallel(n_jobs=8)]: Done   1 out of   8 | elapsed:    0.3s remaining:    2.1s


Test accuracy is : -0.475969322569 and train accuracy is : 0.944697982064


[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.8s finished


Here also we see that the model is performing horribly in predicting the 30secAhead changes. A value of less than shows the model is performing worse than random predictions.

Now we would try to predict just the movement of prices and not its value. The classifier consists of 3 classes:

MOVE UP(+ve), NO MOVE(0), MOVE DOWN(-ve)

# Model Type 2 : Random Forest Classifier
For this model we create the predictor variable showing the movement.

In [8]:
# New binary predictor variables
def create_buckets(data):
    buckets = np.zeros(data.shape[0])
    buckets[data > 0] = 1
    buckets[data == 0] = 0
    buckets[data < 0] = -1
    
    return (buckets)

#30secAhead

y_train_binary = create_buckets (y_train['30secAhead'])
y_test_binary = create_buckets (y_test['30secAhead']) 

rfc0 = RandomForestClassifier(n_jobs=-1, verbose=1)
rfc0.fit(X_train[model_cols_X], y_train_binary)

imporatnce_matrix_rfc0 = pd.DataFrame(0, index = X_train[model_cols_X].columns, columns = ['Importance'])
imporatnce_matrix_rfc0['Importance'] = rfc0.feature_importances_

test_accuracy = rfc0.score(X_test[model_cols_X], y_test_binary)
train_accuracy = rfc0.score(X_train[model_cols_X], y_train_binary)
print (test_accuracy, train_accuracy) 

[Parallel(n_jobs=8)]: Done   1 out of   8 | elapsed:   26.3s remaining:  3.1min
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   26.3s finished
[Parallel(n_jobs=8)]: Done   1 out of   8 | elapsed:    0.7s remaining:    4.9s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    2.2s finished
[Parallel(n_jobs=8)]: Done   1 out of   8 | elapsed:    1.9s remaining:   13.4s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    6.7s finished


0.396001061155 0.995051299088


For further model comparisions we are making a function which can handle the models and produce the results.

accu_above_level: Its a special function which will seperate the predictions setting very high threshold giving us better accuracies on trades we can go in. 

Accu_above_level is a representative of ROC Metric used for 2 class classification. ROC is not being used as the problem is of Multi Class Classification

# Assumption
In addition to that accuracy on Up Move and Down Move is being considered as these are trade producing signals.

Here positive and negative trades refer to Long Side and Short Side trades. Transaction cost is not being considered which will go in when trading stretegies are formed.

# Caution
Though the modeling is being done on just a single set of test and train data but thorough backtessting needs to be done further with monger time period of data.

In [134]:
def RF_model(model, feature_list, verbose = True):
    
    model.fit(X_train[feature_list], y_train_binary)

    test_accuracy = model.score(X_test[feature_list], y_test_binary)
    train_accuracy = model.score(X_train[feature_list], y_train_binary)
    
    if(verbose == True):
        print ('Model is ' + str(model))
        print ('Test accuracy is : ' + str(test_accuracy) + ' and train accuracy is : ' + str(train_accuracy))
    
    imporatnce_matrix = pd.DataFrame(0, index = feature_list, columns = ['Importance'])
    imporatnce_matrix['Importance'] = model.feature_importances_ 
    
    return (test_accuracy, train_accuracy)#, imporatnce_matrix)

# We would like to check the accuracy of predictions at various levels of probability
def pred_matrix(y_test, model):
    prediction = model.predict_proba(X_test[feature_list])
    prediction_matrix = pd.DataFrame(0, index = y_test.index, columns = ['date', 'Date','30secAhead', 'Original_direction','+ve_dir_proba','-ve_dir_proba'])
    prediction_matrix['date'] = y_test['date']
    prediction_matrix['Date'] = y_test['Date']
    prediction_matrix['30secAhead'] = y_test_binary
    prediction_matrix['Original_direction'] = y_test_binary
    prediction_matrix['-ve_dir_proba'] = prediction[:,0]
    prediction_matrix['0_dir_proba'] = prediction[:,1]
    prediction_matrix['+ve_dir_proba'] = prediction[:,2]

    return (prediction_matrix)

def accu_above_level(df ,po_lvl , ne_lvl):
    total = df.shape[0]
    prediction = np.zeros(total)
    prediction[df['+ve_dir_proba'].values > po_lvl] = 1
    prediction[df['-ve_dir_proba'].values > ne_lvl] = -1
    number_of_predictions = np.sum(prediction * prediction)
    cnf_matrix = confusion_matrix(prediction_matrix['Original_direction'], prediction)
    pos_prior, neg_prior = np.sum(cnf_matrix[2,:])/ np.sum(cnf_matrix) , np.sum(cnf_matrix[0,:])/ np.sum(cnf_matrix)
    pos_post, neg_post = cnf_matrix[2,2]/ np.sum(cnf_matrix[:,2]) , cnf_matrix[0,0]/ np.sum(cnf_matrix[:,0])
    print ('Posteriors for +ve, -ve are : %2f and %2f; where as priors were %2f and %2f with %d trades available in test period.' %(pos_post, neg_post, pos_prior, neg_prior, number_of_predictions))
    print ('Number of buy trades: %d and short trades: %d' %(np.count_nonzero(prediction == 1), np.count_nonzero(prediction == -1)))
    return (cnf_matrix)

# RF MODEL Iteration 2:

In [22]:
no_of_features = 8
rfc2 = RandomForestClassifier(n_jobs=-1, verbose=1)
feature_list =imporatnce_matrix_rfc0.sort('Importance', ascending=False).head(no_of_features).index 
print (feature_list)
RF_model(rfc2, feature_list)
# Prepare the prediction matrix
prediction_matrix = pred_matrix(y_test, rfc2,imporatnce_matrix_rfc0)
# Check the accuracy above a particular level
accu_above_level(prediction_matrix,0.7, 0.7)

Index([u'f148', u'f147', u'f146', u'f145', u'f117', u'f116', u'f122', u'f119'], dtype='object')


/Users/k0s00ks/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
[Parallel(n_jobs=8)]: Done   1 out of   8 | elapsed:    5.6s remaining:   39.5s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    5.8s finished
[Parallel(n_jobs=8)]: Done   1 out of   8 | elapsed:    0.1s remaining:    1.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.4s finished
[Parallel(n_jobs=8)]: Done   1 out of   8 | elapsed:    0.3s remaining:    2.3s


Model is RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion=gini, max_depth=None, max_features=auto,
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
            verbose=1)
Test accuracy is : 0.400314886157 and train accuracy is : 0.995584136544


[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.6s finished
[Parallel(n_jobs=8)]: Done   1 out of   8 | elapsed:    0.2s remaining:    1.1s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.4s finished


Posteriors for +ve, -ve are : 0.430657 and 0.440596; where as priors were 0.410869 and 0.385857 with 7956 trades available in test period.
Number of positive trades: 5069 and negative trades: 2887


array([[ 1272, 63560,  2074],
       [  414, 34021,   812],
       [ 1201, 67859,  2183]])

# RF MODEL Iteration 3:

For the model iteration 3 we will select the variables which were highly correlated in the correlation matrix analysed. In addition to this we will take into consideration the variables having high importance in Model 1 of RF Classifiers.

In [56]:
no_of_features = 8
rfc3 = RandomForestClassifier(n_jobs=-1, verbose=1)
# rfc2 = 0
feature_list =[u'bidx0', u'sidx0', u'bidx2', u'sidx2', u'bidx4', u'sidx4', u'f26',
       u'f27', u'bidx1', u'bidx3', u'bsz2_cm', u'bsz3_cm', u'bsz4_cm', u'f82',
       u'f114']
print (feature_list)
RF_model(rfc3, feature_list)
# Prepare the prediction matrix
prediction_matrix = pred_matrix(y_test, rfc3,imporatnce_matrix_rfc0)
# Check the accuracy above a particular level
accu_above_level(prediction_matrix,0.7, 0.7)

[u'bidx0', u'sidx0', u'bidx2', u'sidx2', u'bidx4', u'sidx4', u'f26', u'f27', u'bidx1', u'bidx3', u'bsz2_cm', u'bsz3_cm', u'bsz4_cm', u'f82', u'f114']


[Parallel(n_jobs=8)]: Done   1 out of   8 | elapsed:    7.4s remaining:   51.7s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    7.4s finished
[Parallel(n_jobs=8)]: Done   1 out of   8 | elapsed:    0.3s remaining:    2.2s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.8s finished
[Parallel(n_jobs=8)]: Done   1 out of   8 | elapsed:    0.6s remaining:    4.4s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    1.4s finished


Model is RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion=gini, max_depth=None, max_features=auto,
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
            verbose=1)
Test accuracy is : 0.404836328404 and train accuracy is : 0.970128743884


[Parallel(n_jobs=8)]: Done   1 out of   8 | elapsed:    0.3s remaining:    2.2s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.8s finished


Posteriors for +ve, -ve are : 0.483295 and 0.467973; where as priors were 0.410869 and 0.385857 with 12029 trades available in test period.
Number of positive trades: 5597 and negative trades: 6432


array([[ 3010, 62041,  1855],
       [ 1187, 33023,  1037],
       [ 2235, 66303,  2705]])

We observe that with this rejig we are able to achieve higher number of trades as well as high accuracy on these trades selected. 

# RF MODEL Iteration 4:

For the 4th iteration we will tweak the parameters of the basic model to achieve better results. The model shown here is obtained after a lot of tweaking.

In [57]:
no_of_features = 8
rfc4 = RandomForestClassifier(compute_importances=None, criterion = 'entropy',
                              min_density=None, min_samples_leaf=1, min_samples_split=2,
                              n_estimators=20, oob_score=False, random_state=12345
                              ,n_jobs=-1, verbose=1,max_features = 5)
# rfc2 = 0
feature_list =[u'bidx0', u'sidx0', u'bidx2', u'sidx2', u'bidx4', u'sidx4', u'f26',
       u'f27', u'bidx1', u'bidx3', u'bsz2_cm', u'bsz3_cm', u'bsz4_cm', u'f82',
       u'f114',u'f148', u'f147', u'f146', u'f145', u'f117', u'f116', u'f122', u'f119']
print (feature_list)
RF_model(rfc4, feature_list)
# Prepare the prediction matrix
prediction_matrix = pred_matrix(y_test, rfc4,imporatnce_matrix_rfc0)
# Check the accuracy above a particular level
accu_above_level(prediction_matrix,0.7, 0.7)

[u'bidx0', u'sidx0', u'bidx2', u'sidx2', u'bidx4', u'sidx4', u'f26', u'f27', u'bidx1', u'bidx3', u'bsz2_cm', u'bsz3_cm', u'bsz4_cm', u'f82', u'f114', u'f148', u'f147', u'f146', u'f145', u'f117', u'f116', u'f122', u'f119']


[Parallel(n_jobs=8)]: Done   1 out of   8 | elapsed:   25.7s remaining:  3.0min
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   26.6s finished
[Parallel(n_jobs=8)]: Done   1 out of   8 | elapsed:    0.3s remaining:    2.2s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.9s finished
[Parallel(n_jobs=8)]: Done   1 out of   8 | elapsed:    0.7s remaining:    5.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    1.8s finished


Model is RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion=entropy, max_depth=None, max_features=5,
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            n_estimators=20, n_jobs=-1, oob_score=False,
            random_state=12345, verbose=1)
Test accuracy is : 0.415615123763 and train accuracy is : 0.998619368534


[Parallel(n_jobs=8)]: Done   1 out of   8 | elapsed:    0.3s remaining:    2.3s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    1.0s finished


Posteriors for +ve, -ve are : 0.602390 and 0.532913; where as priors were 0.410869 and 0.385857 with 2364 trades available in test period.
Number of positive trades: 1255 and negative trades: 1109


array([[  591, 65967,   348],
       [  208, 34888,   151],
       [  310, 70177,   756]])

We observe that the model is giving us good accuracy trades but all the models are still overfitting which is a rare phenomenon in Random Forest. 

The number of buy and sell trades

We will move to a more complicated tree algorithm: XGBoost

# Model Type 3: XGBoost Classification

In [ ]:
def XGB_model(model, feature_list, verbose = True):
    
    model.fit(X_train[feature_list], y_train_binary)

    test_accuracy = model.score(X_test[feature_list], y_test_binary)
    train_accuracy = model.score(X_train[feature_list], y_train_binary)
    
    if(verbose == True):
        print ('Model is ' + str(model))
        print ('Test accuracy is : ' + str(test_accuracy) + ' and train accuracy is : ' + str(train_accuracy))
    
    return (test_accuracy, train_accuracy)

# XGB Model 1:

In [133]:
xgb1 = xgb.XGBClassifier(n_estimators=20,
 learning_rate =0.1,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softmax',
 seed=27,
 nthread=-1)

feature_list =[u'bidx0', u'sidx0', u'bidx2', u'sidx2', u'bidx4', u'sidx4', u'f26',
       u'f27', u'bidx1', u'bidx3', u'bsz2_cm', u'bsz3_cm', u'bsz4_cm', u'f82',
       u'f114',u'f148', u'f147', u'f146', u'f145', u'f117', u'f116', u'f122', u'f119']


XGB_model(xgb1, feature_list)
# Prepare the prediction matrix
prediction_matrix = pred_matrix(y_test, xgb1,imporatnce_matrix_rfc0)
# Check the accuracy above a particular level
accu_above_level(prediction_matrix,0.7, 0.7)

Model is XGBClassifier(base_score=0.5, colsample_bytree=0.8, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, n_estimators=200, nthread=-1,
       objective=multi:softprob, seed=27, silent=True, subsample=0.8)
Test accuracy is : 0.420909363538 and train accuracy is : 0.527841296413
Posteriors for +ve, -ve are : 0.555975 and 0.421912; where as priors were 0.410869 and 0.385857 with 58069 trades available in test period.
Number of buy trades: 3707 and short trades: 54362


array([[22936, 42706,  1264],
       [10396, 24469,   382],
       [21030, 48152,  2061]])

We observe that the trades produced are very high with a decent number of accuracy. In addition to this the model is not overfitting; which is a good thing as compared to our previous models.

# XGB MODEL 2: 
We shift to a simpler model with more number of trees.

In [136]:
xgb2 = xgb.XGBClassifier(n_estimators=200,
 learning_rate =0.01,
 objective= 'multi:softmax')

feature_list =[u'bidx0', u'sidx0', u'bidx2', u'sidx2', u'bidx4', u'sidx4', u'f26',
       u'f27', u'bidx1', u'bidx3', u'bsz2_cm', u'bsz3_cm', u'bsz4_cm', u'f82',
       u'f114',u'f148', u'f147', u'f146', u'f145', u'f117', u'f116', u'f122', u'f119']


XGB_model(xgb2, feature_list)
# Prepare the prediction matrix
prediction_matrix = pred_matrix(y_test, xgb2)
# Check the accuracy above a particular level
accu_above_level(prediction_matrix,0.7, 0.7)

Model is XGBClassifier(base_score=0.5, colsample_bytree=1, gamma=0, learning_rate=0.05,
       max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=200,
       nthread=-1, objective=multi:softprob, seed=0, silent=True,
       subsample=1)
Test accuracy is : 0.432933862373 and train accuracy is : 0.457088248238
Posteriors for +ve, -ve are : 0.621339 and 0.425828; where as priors were 0.410869 and 0.385857 with 48481 trades available in test period.
Number of buy trades: 478 and short trades: 48003


array([[20441, 46329,   136],
       [ 8527, 26675,    45],
       [19035, 51911,   297]])

In this model the test accuracy has increased a little but at the same time we notice that most of the trades produced are negative trades. This skew in the predictions need to be handled.

In [143]:
xgb3 = xgb.XGBClassifier(n_estimators=200,
 learning_rate =0.01,
 gamma =1,
 objective= 'multi:softmax',
 seed=27,
 nthread=-1)

feature_list =[u'bidx0', u'sidx0', u'bidx2', u'sidx2', u'bidx4', u'sidx4', u'f26',
       u'f27', u'bidx1', u'bidx3', u'bsz2_cm', u'bsz3_cm', u'bsz4_cm', u'f82',
       u'f114',u'f148', u'f147', u'f146', u'f145', u'f117', u'f116', u'f122', u'f119']


XGB_model(xgb3, feature_list)
# Prepare the prediction matrix
prediction_matrix = pred_matrix(y_test, xgb3)
# Check the accuracy above a particular level
accu_above_level(prediction_matrix,0.5, 0.5)

Model is XGBClassifier(base_score=0.5, colsample_bytree=1, gamma=1, learning_rate=0.01,
       max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=200,
       nthread=-1, objective=multi:softprob, seed=27, silent=True,
       subsample=1)
Test accuracy is : 0.439300791252 and train accuracy is : 0.446893147753
Posteriors for +ve, -ve are : 0.589481 and 0.590368; where as priors were 0.410869 and 0.385857 with 7091 trades available in test period.
Number of buy trades: 1464 and short trades: 5627


array([[ 3322, 63208,   376],
       [  695, 34327,   225],
       [ 1610, 68770,   863]])

In [147]:
accu_above_level(prediction_matrix,0.45, 0.44)

Posteriors for +ve, -ve are : 0.579422 and 0.494019; where as priors were 0.410869 and 0.385857 with 41212 trades available in test period.
Number of buy trades: 6434 and short trades: 34778


array([[17181, 47938,  1787],
       [ 6204, 28124,   919],
       [11393, 56122,  3728]])

At this level with similar number of trades accuracy of short trades has increased. In addition to this we are getting good number of buy trades too. 